### This is used to test the Option-shortening Workflow

In [1]:
from src.general import extract_mcq_components, extract_correct_answer_letter

In [ ]:
from src.option_shortening_helper import identify_longer_options, syntactic_analysis, calculate_length_range, generate_candidate_short_options, select_best_candidate

In [2]:
mcq = '''How do the id, ego, and superego interact in Freud's personality model?

A) The ego prioritizes the fulfillment of the id's desires.

B) The superego suppresses the ego entirely to enforce social rules.

C) The id harmonizes completely with the superego to reduce anxiety.

D) The ego balances the instinctive drives of the id with the moral standards of the superego.'''


In [4]:
mcq_answer = "D) The ego balances the instinctive drives of the id with the moral standards of the superego."

In [5]:
question, options = extract_mcq_components(mcq)

In [6]:
longer_option_index, longer_option_text = identify_longer_options(options)

In [ ]:
longer_option_text

In [ ]:
if longer_option_index:
    print ("The question contains longer options that may need shortening")
else:
    print ("No longer options detected, no shortening needed")
    exit()

In [9]:
invocation_id = "test20250910"

In [ ]:
identified_rule_meta = await syntactic_analysis(
    invocation_id=invocation_id,
    model="gpt-4o",
    temperature=0.3,
    question_stem=question,
    options=options)

In [ ]:
identified_rule = identified_rule_meta.get("syntactic_rule", "")
print(f"Identified syntactic rule: {identified_rule}")


In [12]:
min_length, max_length = calculate_length_range(options)

In [ ]:
print(f"Length range for options: min_length={min_length}, max_length={max_length}")

In [ ]:
shortened_options_candidates_meta = await generate_candidate_short_options(
        invocation_id=invocation_id,
        model="gpt-4o",
        options=options,
        option_to_shorten=longer_option_text,
        syntactic_rule=identified_rule,
        min_target=min_length,  
        max_target=max_length,
        table_name="candidate_shortening_metadata",
        database_file= '../database/mcq_metadata.db'
    )  # List of lists, each inner list contains candidates for one longer option

In [14]:
shortened_options_candidates = shortened_options_candidates_meta.get("candidates", [])

In [ ]:
shortened_options_candidates

In [ ]:
candidate_selection_meta = await select_best_candidate(
    invocation_id=invocation_id,
    model="gpt-4o",
    options=options,
    option_to_shorten=longer_option_text,
    syntactic_rule=identified_rule,
    min_target=min_length,
    max_target=max_length,
    candidates=shortened_options_candidates,
    table_name="candidate_selection_metadata",
    database_file= '../database/mcq_metadata.db'
)  # Returns the best candidate or None    

In [18]:
best_candidate = candidate_selection_meta.get("best_candidate", None)

In [ ]:
best_candidate

In [ ]:
# test the whole pipeline:
from src.option_shortener_workflow import check_and_shorten_long_option 


In [ ]:
updated_mcq, updated_mcq_answer, token_usage = await check_and_shorten_long_option(
    invocation_id="test20250910",
    mcq=mcq,    
    mcq_answer=mcq_answer,
    model="gpt-4o",
    )

In [8]:
if token_usage:
    print("True")
else:
    print("False")

True


In [ ]:
print("Updated MCQ:")
print(updated_mcq)   
print("Updated MCQ Answer:")   
print(updated_mcq_answer)
print("Token usage:")
print(token_usage)

In [7]:
mcq2 = '''What function did early exhibitors serve in the showcasing of movies in theaters?
    \nA) They created and pre-recorded the content displayed in theaters.
    \nB) They determined the arrangement of different elements in the film program.
    \nC) They frequently participated in live performances.
    \nD) They offered guidance to filmmakers regarding suitable movie content.'''

In [12]:
mcq_updated_2, token_usage_2 = await check_and_shorten_long_option(
    invocation_id="test20250910",
    mcq=mcq2,    
    model="gpt-4o",
    )

In [ ]:
if token_usage_2:
    print("True")
else:
    print("False")